# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Загрузка файла с данными и изучие общей информации. 

In [37]:
# <импорт библиотеки pandas>
import pandas as pd

In [38]:
# чтение файла
clien_data = pd.read_csv('data.csv')

# получение общей информации о данных в таблице clien_data
clien_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [39]:
# получение первых 10 строк таблицы clien_data
clien_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов, тип данных у каждого столбца числовой и текстовый.

Подробно разберём, какие в clien_data столбцы и какую информацию они содержат:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропуски значения.

### Вывод

Каждая строка содержит данные о потенциальных клиентах банках и даннных из анкеты (семейное положение, образование, трудовая занятости, целевой назначение)

Проблемы которые нужно решить:
- пропуски значений.
- разный регистор в столбце education
- отрицательные данные в столбце days_employed

Пропуски значений могли появитсья из-за ошибок при вводе данных. 

Для проверки рабочих гипотез особенно ценны столбцы "children" и "family_status"

### Шаг 2. Предобработка данных

In [40]:
# суммарное количество пропусков
clien_data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [41]:
# проверка общих данных по clien_data методом describe
clien_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [42]:
# в таблице были видны отрицательные значения в столбце days_employed и уберем из методом abs
clien_data['days_employed'] = clien_data['days_employed'].abs()

In [43]:
# поччитаем колличесво записей по столбцу children
clien_data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Обнаружены отрицательные значения в столюцах которые не могут быть отрицательными (children, days_employed)
Также встречаются клиенты у которых 20 детей. Это конечно теоретически возможно если рассматривать приемные семьи, но в рамках исследования предположим, что это ошибка в записи и заменим значения на 2 и уберем отрицательные значения


In [44]:
# заменим значения 20 на 2 мотодом replace и уберем отрицательные значения методом abs
clien_data['children'] = clien_data['children'].replace(20, 2).abs()

In [45]:
# выполним проверку
clien_data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [46]:
# проверка уникальных значний  dob_years
clien_data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Пустые значения свидетельствуют о том что у части клиентов не заполнены данные о трудовой деятельности

In [47]:
#заменим нулевое значение в dob_years на среднее значение методом replace
age_mean = int(clien_data['dob_years'].mean())
clien_data['dob_years'] = clien_data['dob_years'].replace(0, age_mean)

In [48]:
# проверка уникальных значний  dob_years
clien_data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

In [49]:
# подсчет по половому признаку
clien_data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [50]:
# заменяем аномальное значеинена в gender любой пол так как запись одна и не повлияет на общую картину. 
clien_data['gender'] = clien_data['gender'].replace('XNA', 'F')

### Обработка пропусков

In [51]:
# расчет медианных значений по категориям для income_groupe
income_groupe = clien_data.groupby('income_type')
income_groupe_mean = income_groupe.total_income.agg(['median'])
income_groupe_mean

,median
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


In [52]:
# расчет медианных значений по категориям для days_employed
employed_groupe = clien_data.groupby('income_type')
employed_groupe_mean = income_groupe.days_employed.agg(['median'])
employed_groupe_mean

,median
income_type,
безработный,366413.652744
в декрете,3296.759962
госслужащий,2689.368353
компаньон,1547.382223
пенсионер,365213.306266
предприниматель,520.848083
сотрудник,1574.202821
студент,578.751554


In [53]:
#замена процещенные значений в total_income медианными значениям по группам профессий 
#посколько разница между максимальными и минимальными данными очень велика
clien_data["total_income"] = clien_data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))

In [54]:
#замена процещенные значений в days_employed медианными значениям по группам профессий 
clien_data["days_employed"] = clien_data.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.median()))

In [55]:
# выполним проверку на пропущенные значения
clien_data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

В данных встречались отрицательные значения и пропуски. Чтобы убрать отрицательный опыт работы возвел все элементы в абсолютное значение и затем посчитал медиану к столбзам с пропусками при этом группируя по профессиям, чтобы медианнны значения соответсоввали выборке по типу занятости.
Также встречались клиенты, у которых возраст был равен 0, такие данные заменили на средний показатель, так как такиз записей было мало

### Замена типа данных

In [56]:
# заменяем вещественный тип данных на целочисленный методом astype('int')
clien_data['days_employed'] = clien_data['days_employed'].astype('int')
clien_data['total_income'] = clien_data['total_income'].astype('int')
clien_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


### Вывод

Чтобы привести данные в более нагляный вид заменили тип данных в days_employed и total_income на целочисленный для этого выбрал метод astype('int')

### Обработка дубликатов

In [57]:
# проверка на уникальные значения
clien_data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [58]:
# удалим дубликаты регистра в столбце образование переведя
# все значения в нижний регистр так как дубликаты появились из-за онибок в записи
clien_data['education'] = clien_data['education'].str.lower()

# проверка на уникальные значения
clien_data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [59]:
# проверка уникальных значний family_status
clien_data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [60]:
# проверка на уникальные значения income_type
clien_data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [61]:
# получение суммарного количества дубликатов в таблице df
clien_data.duplicated().sum()

71

In [62]:
# <удаление всех дубликатов из таблицы df 
clien_data = clien_data.drop_duplicates().reset_index(drop=True)

In [63]:
# проверка на отсутствие дубликатов
clien_data.duplicated().sum()

0

### Вывод

Дубликаты могли появиться вследствие сбоя в записи данных, а также ошибках ввода данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».



### Лемматизация

In [64]:
# проверка на уникальные значения purpose
purpose_unique = clien_data['purpose'].unique()
purpose_unique

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [65]:
# Импортируем pymystem3:
from pymystem3 import Mystem
from collections import Counter

In [66]:
m = Mystem()

# лемматизация purpose
str_purpose_unique = (' '.join(purpose_unique))
lemmas = m.lemmatize(str_purpose_unique)

# подсчёт количества уникальных.
сounter=Counter(lemmas)
сounter

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

In [ ]:
# добавим столбец 'purpose_lemm', чтобы увидеть леммы целей критования

clien_data['purpose_lemm'] = clien_data['purpose'].apply(m.lemmatize)

In [ ]:
# напишем функую, которая запишет цель кредита одним словом, для этого выделим наиболее частые категории
def purpose_category(clien_data):
    for row in clien_data['purpose_lemm']:
        if row == 'автомобиль':
            return 'автомобиль'
        if row == 'недвижимость' or row == 'жилье':
            return 'недвижимость'
        if row == 'образование':
            return 'образование'
        if row == 'свадьба':
            return 'свадьба'

In [ ]:
# добавляем столбец с категорией кредитования. 
# Исмользуем метоб .apply чтобы применить функцию 
clien_data['purpose_category'] = clien_data.apply(purpose_category, axis=1)
clien_data.head(5)

In [ ]:
# проверка на наличие пропусков в созданых столбцах
clien_data.isnull().sum()

### Вывод

Проведя лемматизацию целей кредитов выделил цели кредитования. Добавил столбей с целью кредита, записанной в обно слово, для удобства анализа. 
Наиболее популярные цели:
- Недвижимость
- Автомобиль
- Образование
- Свадьба

Выбор целевых категорий крелдитов проводился на оснвание анализа колличесва лемм. Так, нииболее пополулярные были:
- жилье
- автомобиль
- недвижимость
- свадьба
- образование
- коммерческая

Жилье, недвижимость и коммерческую недвижимость объединил в одну группу Недвижимость

### Категоризация данных

In [ ]:
# напишем функцию для заполения столбца have_children
def h_children(row):
        if row == 0:
            return 'Детей нет'
        if row == 1:
            return '1 ребенок в семье'
        if row == 2:
            return '2 ребенок в семье'
        if row == 3:
            return '3 ребенок в семье'
        else:
            return 'В семье более 3 детей'
        
clien_data['have_children'] = clien_data['children'].apply(h_children)

In [ ]:
# посчитаем колличество клиентов каждой группе have_children
count_have_ch = clien_data.pivot_table(index=['have_children'], values='children', aggfunc='count')
count_have_ch

In [ ]:
# посчитаем кол-во клиентов в каждой круппе в зависимости от семейного полождения
data_pivot_famaly_count = clien_data.pivot_table(index=['family_status'], values='family_status_id', aggfunc='count')
data_pivot_famaly_count

In [ ]:
# посчитаем кол-во клиентов в каждой круппе в зависимости от цели кредита
data_pivot_purpose_count = clien_data.pivot_table(index=['purpose_category'], values='purpose', aggfunc='count')
data_pivot_purpose_count

In [ ]:
# посчитаем кол-во клиентов в каждой круппе в зависимости от категории дохода 
profit = pd.cut(clien_data['total_income'], [ 20000, 60000, 100000, 140000, 180000, 999999999])
data_pivot_income_count = clien_data.pivot_table(index=[profit], values='total_income', aggfunc='count')
data_pivot_income_count

In [ ]:
# расчет медианных значений по категориям для income_groupe
income_groupe = clien_data.groupby('income_type')
income_groupe_mean = income_groupe.total_income.agg(['min', 'max', 'median', 'count'])
income_groupe_mean

### Шаг 3. Ответ на поставленные вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
#  на основание ранее сознанных категорий по наличию детеой создадим своднубю даблицу с колличеством просрочек
data_pivot = clien_data.pivot_table(index=['have_children'], values='debt', aggfunc='sum')

# объединим две таблицы для рассчета доли кредитов с просрочкой в каждой из групп
final_pivot_children = data_pivot.join(count_have_ch)
final_pivot_children['defolt_percent'] = (final_pivot_children['debt'] / final_pivot_children['children'])
final_pivot_children.sort_values(by='defolt_percent', ascending=False).style.format({'defolt_percent': "{:.1%}"})

### Вывод

Как видно семьи у которых нет детей рехе добускают просрочку. Доля таких кредитов у семей без детей 7,5%.
С появлением первого ребенка доля кредитов с просрочной увеличивается до 9,2%. вероятней всего это связано с недостатком дохода одного из супругов, так как общей семейный доход снижается.
Стоит отметить, что доля проблемных кредитов у семей с 3 детьми и более равно 8%

Из данной выборки можно сделать вывод, что предпочтительней клиенты без детей, а клиент с детьми более рискованные, при этом с ростом коллическа детей риски снижаются. Соответвенно при расчете процентной ставки стоит учитывать данный фактор.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#  создадим своднубю даблицу с колличеством просрочек в зависимости от семейного положения
data_pivot_famaly = clien_data.pivot_table(index=['family_status'], values='debt', aggfunc='sum')

# объединим две таблицы для рассчета доли кредитов с просрочкой в каждой из групп
final_pivot_famaly = data_pivot_famaly.join(data_pivot_famaly_count)
final_pivot_famaly['defolt_percent'] = (final_pivot_famaly['debt'] / final_pivot_famaly['family_status_id'])
final_pivot_famaly.sort_values(by='defolt_percent', ascending=False).style.format({'defolt_percent': "{:.1%}"})

### Вывод

Наибольшая доля просроченных кредитов у  одиноких клиентов, которые не были в браке 9,8%.
Клиенты в брае более надежные, доля проблемных клиентов 7,5%

Клиенты в разводе еще более превлекательные клиенты доля просроченных кредитов всего 7,1%, при этом клеинты в браке домускают просрочку в 7,5% кредитов 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#создадим своднубю даблицу с колличеством просрочек в зависимости от категории дохода
data_pivot_income = clien_data.pivot_table(index=[profit], values='debt', aggfunc='sum')

# объединим две таблицы для рассчета доли кредитов с просрочкой в каждой из групп
final_pivot_income = data_pivot_income.join(data_pivot_income_count)
final_pivot_income['defolt_percent'] = (final_pivot_income['debt'] / final_pivot_income['total_income'])
final_pivot_income.sort_values(by='defolt_percent', ascending=False).style.format({'defolt_percent': "{:.1%}"})

### Вывод

Доля проблемных кредитов у людей с доходом от 140000 до 180000 составляет 8,7%, при этом доля проблемных кредитов у клиенты с доходом от 20000 до 60000 составляет 6,1%.

Вероятно причана в сумме кредита, клиенты  с небольшим доходом берут не больные кредиты на покупку бытовой техники, которые плегче выплачивать так ка срок не большой.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#создадим своднубю даблицу с колличеством просрочек в зависимости от цели кредита
data_pivot_purpose = clien_data.pivot_table(index=['purpose_category'], values='debt', aggfunc='sum')

# объединим две таблицы для рассчета доли кредитов с просрочкой в каждой из групп
final_pivot_purpose = data_pivot_purpose.join(data_pivot_purpose_count)
final_pivot_purpose['defolt_percent'] = (final_pivot_purpose['debt'] / final_pivot_purpose['purpose'])
final_pivot_purpose.sort_values(by='defolt_percent', ascending=False).style.format({'defolt_percent': "{:.1%}"})

### Вывод

Наибольшая доля проблемных кредитов приходится на покупку автомобилей, 9,4%% таких кредитов выходит на просрочку.
Далее по величине риска идут кредиты на обдразование 9,2%

Наиболее привлекательные кредиты на покупку недвиждимости 7,2% таких клиетовы допуцскают просрочки

### Шаг 4. Общий вывод

В ходе работы получилось обределить наиболее привлекательных клиентов и ответить на поставленные вопросы.

Предпочтительней клиенты без детей, а клиент с детьми более рискованные, при этом с ростом коллическа детей риски снижаются. Соответвенно при расчете процентной ставки стоит учитывать данный фактор.
Клеинты в браке домускают просрочку в 7,5% кредитов.

Самые низкорискованные кредиты - на покупку нежвижимости доля риска 7.2%, что соответвует клиентам с доходом от 180000 (7,4%)


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [X]  в каждом этапе есть выводы;
- [x]  есть общий вывод.